## GLUE dataset과 Huggingface

### GLUE Benchmark Dataset

Pretrained model의 성능을 측정하기 위해 최근은 SQuAD 등 기존에 유명한 데이터셋 한 가지만 가지고 성능을 논하는 것이 아니라, classification, summarization, reasoning, Q&A 등 NLP 모델의 성능을 평가할 수 있는 다양한 task를 해당 모델 하나만을 이용해 모두 수행해 보면서 종합적인 성능을 논하는 것이 일반화되었습니다.  
  
그중 NLP 모델의 성능을 측정하기 위한 데이터셋으로 최근 활용되는 대표적인 것 중에 General Language Understanding Evaluation(GLUE) benchmark Dataset이 있습니다. 총 10가지 데이터셋이 있습니다. 각각의 개요는 다음과 같습니다.  
  
* CoLA : 문법에 맞는 문장인지 판단
* MNLI : 두 문장의 관계 판단(entailment, contradiction, neutral)
* MNLI-MM : 두 문장이 안 맞는지 판단
* MRPC : 두 문장의 유사도 평가
* SST-2 : 감정분석
* STS-B : 두 문장의 유사도 평가
* QQP : 두 질문의 유사도 평가
* QNLI : 질문과 paragraph 내 한 문장이 함의 관계(entailment)인지 판단
* RTE : 두 문장의 관계 판단(entailment, not_entailment)
* WNLI : 원문장과 대명사로 치환한 문장 사이의 함의 관계 판단  
  
GLUE 홈페이지에는 위 10가지 task에 대한 상세한 설명, 그리고 Leaderboard를 운영하고 있습니다. 한 가지 task에만 최적화된 모델이 아니라, 다양한 형태의 문제를 골고루 잘 푸는 모델을 찾기 위한 노력이 계속되고 있습니다.

### Huggingface가 제공하는 GLUE task 예제 코드
Huggingface와 같은 NLP framework는 해당 framework를 활용하여 새로 만들어진 모델의 성능을 빠르게 평가해 볼 수 있도록 하는 예제 코드를 제공하고 있습니다. 이런 예제 코드가 없다면 모델을 새로 만들 때마다 그 성능을 비교 측정해 보기 위한 작업이 너무나 번거롭겠죠? 오늘은 우선 Huggingface의 예제 코드를 들여다보는 것으로부터 시작해 보겠습니다.  
우선 소스코드 프로젝트를 통해 다시 설치해 봅시다. 설치하지 않으면 오류가 발생하므로 꼭 설치해 주세요.

In [10]:
# $ cd ~/aiffel && git clone https://github.com/huggingface/transformers.git
# $ cd transformers && pip install -e .
# $ pip install datasets

# $ cd ~/aiffel/transformers
# $ python examples/tensorflow/text-classification/run_glue.py \
# 	--model_name_or_path bert-base-cased \
# 	--task_name mrpc \
# 	--output_dir ./models/mrpc \
# 	--overwrite_output_dir \
# 	--do_train \
# 	--do_eval \
# 	--num_train_epochs 1 \
# 	--save_steps 20000

위 코드는 10가지 GLUE task 중 'mrpc' task를 수행하는 예제 코드입니다. 이 코드는 Huggingface의 framework 기반으로 BERT `bert-base-cased`을 활용하여 'mrpc' task를 수행합니다. 만약 `task_name` 및 다른 파라미터를 적절히 변경한 후 수행하면 다른 GLUE task도 간단히 수행해 볼 수 있을 것입니다. model도 다양하게 바꾸어 보면서 수월하게 수행 가능할 것입니다. 이 예제만으로도 NLP framework의 강력함을 손쉽게 느껴볼 수 있을 것입니다.

## 커스텀 프로젝트 제작 (1) Processor

### mrpc 데이터셋 분석
본격적으로 Huggingface framework를 활용해 봅시다. 언제나 그렇듯, 프로젝트를 수행하기 위한 첫 단계는 데이터를 분석하는 것입니다.

In [11]:
import os
import numpy as np
from argparse import ArgumentParser
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

GLUE 데이터셋은 홈페이지에서 원본을 다운로드할 수도 있지만, 이번에는 tensorflow_datasets에서 제공하는 것을 이용해 보겠습니다.

In [12]:
data, info = tfds.load('glue/mrpc', with_info=True)
info.splits['train'].num_examples

INFO:absl:Load dataset info from /aiffel/tensorflow_datasets/glue/mrpc/1.0.0
INFO:absl:Reusing dataset glue (/aiffel/tensorflow_datasets/glue/mrpc/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /aiffel/tensorflow_datasets/glue/mrpc/1.0.0


3668

`data`는 `tf.data.Dataset`을 상속받은 클래스의 형태일 것입니다. 우선 1개의 데이터만 가져다가 어떻게 생겼는지 확인해 봅시다.

In [13]:
data['train'].take(1)

<TakeDataset shapes: {idx: (), label: (), sentence1: (), sentence2: ()}, types: {idx: tf.int32, label: tf.int64, sentence1: tf.string, sentence2: tf.string}>

데이터셋 안에 어떤 항목이 정의되어 있는지 확인할 수 있었습니다. 실제 내용도 한번 확인해 볼까요?

In [14]:
examples = data['train'].take(1)
for example in examples:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['label']
    print(sentence1)
    print(sentence2)
    print(label)

tf.Tensor(b'The identical rovers will act as robotic geologists , searching for evidence of past water .', shape=(), dtype=string)
tf.Tensor(b'The rovers act as robotic geologists , moving on six wheels .', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


### Processor의 활용
우리는 지난 시간에, `Huggingface transformers`에서 task별로 데이터셋을 가공하는 일반적인 클래스 구조인 `Processor`에 대해 다룬 바 있습니다.  
  
아래는 추상클래스인 `Processor`를 한번 상속받은 후, Sequence Classification task를 수행하는 모델의 `Processor` 추상클래스인 `DataProcessor`입니다.

In [16]:
class DataProcessor:
    """ sequence classification을 위해 data를 처리하는 기본 processor"""
    
    def get_example_from_tensor_dict(self, tensor_dict):
        """
        tensor dict에서 example을 가져오는 메소드
        """
        raise NotImplementedError()
        
    def get_train_examples(self, data_dir):
        """train data에서 InputExample 클래스를 가지고 있는 것들을 모으는 메소드"""
        raise NotImplementedError()
        
    def get_test_examples(self, data_dir):
        """test data에서 InputExample 클래스를 가지고 있는 것들을 모으는 메소드"""
        raise NotImplementedError()
        
    def get_labels(self):
        """data set에 사용되는 라벨들을 리턴하는 메소드"""
        raise NotImplementedError()
        
    def tfds_map(self, example):
        """
        tfdf(tensorflow-datasets)에서 불러온 데이터를 DataProcessor에 알맞게 가공해주는 메소드
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example
    
    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """ tab으로 구분된 .tsv파일을 읽어들이는 클래스 메소드"""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

아직은 추상클래스 상태이기 때문에 그대로 사용하면 NotImplementedError를 발생시키는 메소드들이 포함되어 있습니다. 이 메소드들을 오버라이드해야 실제 사용 가능한 클래스가 얻어지겠죠?  
  
아래는 'mrpc' 원본 데이터셋을 처리하여 모델에 입력할 수 있도록 정리해 주는 `MrpcProcessor` 클래스입니다.

In [17]:
class MrpcProcessor(DataProcessor):
    """Oricessor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence1"].numpy().decode("utf-8"),
            tensor_dict["sentence2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )
    
    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self, read_tsv(os.path.join(data_dir, "train.tsv")), "train")
    
    def get_dev_examples(telf, data_dir):
        """See base class."""
        return self.create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")
    
    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self, read_tsv(os.path.join(data_dir, "test.tsv")), "test")
    
    def get_labels(self):
        """See base class."""
        return ["0", "1"]
    
    def create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

이것만으로는 클래스 구조와 메커니즘이 눈에 잘 안 들어오죠? 여기서 우선 주목해야 할 메소드는 `get_example_from_tensor_dict()`입니다. 실제로 이 메소드가 어떤 역할을 하게 되는지 살펴봅시다.

In [18]:
processor = MrpcProcessor()
examples = data['train'].take(1)

for example in examples:
    print('-------원본데이터-------')
    print(example)
    example = processor.get_example_from_tensor_dict(example)
    print('-------가공데이터-------')
    print(example)

-------원본데이터-------
{'idx': <tf.Tensor: shape=(), dtype=int32, numpy=1680>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'sentence1': <tf.Tensor: shape=(), dtype=string, numpy=b'The identical rovers will act as robotic geologists , searching for evidence of past water .'>, 'sentence2': <tf.Tensor: shape=(), dtype=string, numpy=b'The rovers act as robotic geologists , moving on six wheels .'>}
-------가공데이터-------
InputExample(guid=1680, text_a='The identical rovers will act as robotic geologists , searching for evidence of past water .', text_b='The rovers act as robotic geologists , moving on six wheels .', label='0')


원본데이터: idx, label, sentence1, sentence2  
가공데이터: guid, text_a, text_b, label

원본과 비교해보니 Processor가 하는 역할이 무엇인지 좀 더 명확해지시나요? 한마디로 요약하자면 Processor는 'Raw Dataset를 Annotated Dataset으로 변환'하는 역할을 합니다. 항목별로 `text_a`, `text_b`, `label` 등의 annotation이 포함된 `InputExample`로 변환되어 있음을 알 수 있습니다.  
다음 코드는 `tfds_map()` 메소드를 활용한 경우입니다.

In [20]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=1680, text_a='The identical rovers will act as robotic geologists , searching for evidence of past water .', text_b='The rovers act as robotic geologists , moving on six wheels .', label='0')


이전과 별다른 차이는 없습니다. `tfds_map`는 label을 가공하는 메소드인데, 이미 숫자로 잘 가공된 label이기 때문에 특별한 변화가 없습니다.  
실제 label을 확인하여 Binary Classification 문제로 잘 정의되고 있는지 확인해 봅시다.

In [21]:
label_list = processor.get_labels()
label_list

['0', '1']

In [22]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

{'0': 0, '1': 1}

## 커스텀 프로젝트 제작 (2) Tokenizer와 Model

Processor를 통해 Framework을 활용하여 데이터셋을 가공하는 작업을 잘 진행했다면 이미 절반 이상 진행한 것이나 마찬가지입니다. NLP 모델링의 핵심을 이루는 Tokenizer와 Model은 framework에서 이미 잘 만들어져 있는 것을 쉽게 가져다 쓸 수 있기 때문입니다.  
  
그럼 이전 스텝에서 만든 `MRPCProcessor` 클래스와 framework를 결합시켜 나가는 과정을 진행해 보겠습니다. 우선 아래와 같이 `tokenizer`와 `model`을 간단히 생성합니다.

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


이제 processor와 tokenizer, 원본 데이터셋을 결합하여 model에 입력할 데이터셋을 생성해 보겠습니다.

In [37]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

In [38]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

`_glue_convert_examples_to_features()` 함수는 processor가 생성한 example을  
`tokenizer`로 인코딩하여 feature로 변환하는 역할을 합니다. 이후  
`tf_glue_convert_examples_to_features()` 함수는 내부적으로  
`_glue_convert_examples_to_features()`를 호출해서 얻은 feature를 바탕으로  
`tf.data.Dataset`을 생성하여 리턴합니다.

In [39]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)

Using label list ['0', '1']
*** Example ***
guid: 1680
features: InputFeatures(input_ids=[101, 1996, 7235, 9819, 2097, 2552, 2004, 20478, 21334, 2015, 1010, 6575, 2005, 3350, 1997, 2627, 2300, 1012, 102, 1996, 9819, 2552, 2004, 20478, 21334, 2015, 1010, 3048, 2006, 2416, 7787, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0,

`tf_glue_convert_examples_to_features()` 함수가 최종적으로 모델에 전달될
`tf.data.Dataset` 인스턴스를 생성합니다. 위 코드는 그렇게 생성된 학습 단계 데이터셋
`train_dataset`입니다.

In [41]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101,  1996,  7235,  9819,  2097,  2552,  2004, 20478, 21334,
        2015,  1010,  6575,  2005,  3350,  1997,  2627,  2300,  1012,
         102,  1996,  9819,  2552,  2004, 20478, 21334,  2015,  1010,
        3048,  2006,  2416,  7787,  1012,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  

그럼 전체 데이터셋을 구성해 보겠습니다.

In [48]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

Using label list ['0', '1']
*** Example ***
guid: 1680
features: InputFeatures(input_ids=[101, 1996, 7235, 9819, 2097, 2552, 2004, 20478, 21334, 2015, 1010, 6575, 2005, 3350, 1997, 2627, 2300, 1012, 102, 1996, 9819, 2552, 2004, 20478, 21334, 2015, 1010, 3048, 2006, 2416, 7787, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0,

In [49]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

Using label list ['0', '1']
*** Example ***
guid: 3155
features: InputFeatures(input_ids=[101, 1996, 2265, 1005, 1055, 8503, 5360, 2353, 1011, 4284, 16565, 2566, 3745, 2011, 1037, 10647, 1012, 102, 1996, 2194, 2056, 2023, 19209, 16565, 2011, 1037, 10647, 1037, 3745, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0

In [50]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test'], tokenizer, max_length=128, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16)

Using label list ['0', '1']
*** Example ***
guid: 163
features: InputFeatures(input_ids=[101, 6661, 1999, 8670, 2020, 2091, 1015, 1012, 1019, 3867, 2012, 16923, 7279, 3401, 2011, 16087, 2692, 13938, 2102, 1010, 2125, 1037, 2659, 1997, 17943, 2361, 1010, 1999, 1037, 3621, 6428, 3452, 2414, 3006, 1012, 102, 6661, 1999, 8670, 2020, 2091, 2093, 3867, 2012, 13913, 1011, 1015, 1013, 1018, 7279, 3401, 2011, 5641, 22394, 13938, 2102, 1010, 2125, 1037, 2659, 1997, 17943, 7279, 3401, 1010, 1999, 1037, 6428, 3006, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## 커스텀 프로젝트 제작 (3) Train/Evaluation과 Test
데이터셋과 모델이 준비되었으므로 이후의 과정은 그동안 익숙하게 진행했던 것과 크게 다르지 않게 진행할 수 있습니다.

### tf.keras.model 을 활용한 학습
`model.fit()`을 이용한 모델학습을 진행해 봅시다.

In [51]:
num_calsses = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [52]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


학습을 진행해 봅시다. 이 학습은 이미 잘 훈련된 BERT 모델을 가져다가 fine-tuning하는 작업임을 기억합시다.  
  
편의상 2 Epoch만 진행한 후의 성능을 체크해 봅시다.

In [53]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model.fit(train_dataset_batch, epochs=2, steps_per_epoch=115,
         validation_data=validation_dataset_batch)

Epoch 1/2
115/115 [==============================] - 117s 851ms/step - loss: 0.6206 - acc: 0.6817 - val_loss: 0.6491 - val_acc: 0.7034
Epoch 2/2
115/115 [==============================] - 94s 818ms/step - loss: 0.5789 - acc: 0.7133 - val_loss: 0.5408 - val_acc: 0.7304


In [54]:
result = model.evaluate(test_dataset_batch)
print(result)

108/108 [==============================] - 60s 552ms/step - loss: 0.2479 - acc: 0.9345
[0.24785485863685608, 0.9344927668571472]


In [55]:
import os
output_dir = os.getenv('HOME') + '/aiffel/transformers'
output_eval_file = os.path.join(output_dir, "eval_results.txt")

with open(output_eval_file, "w") as writer:
    for i, v in enumerate(result):
        if i == 0:
            writer.write("Loss = %f\t" %(v))
        if i == 1:
            writer.write("Accuracy = %f\n" %(v))
print("완료")

# 파일에 쓴 테스트 결과 확인
! cat ~/aiffel/transformers/eval_results.txt

완료
Loss = 0.247855	Accuracy = 0.934493


### TFTrainer를 활용한 학습
이번에는 Huggingface의 `TFTrainer`를 활용해 학습을 진행해 봅시다.  
  
이전 노드에서 살펴본 것처럼 `TFTrainer`를 활용하기 위해서는 `TFTrainingArguments`에 학습 관련 설정을 미리 지정해 두어야 합니다.

In [69]:
# TFTrainer를 활용하는 형태로 모델 재생성
from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
    glue_compute_metrics,
    glue_convert_examples_to_features,
    glue_output_modes,
    glue_processors,
    glue_tasks_num_labels,
)

output_dir = os.getenv('HOME') + '/aiffel/transformers'

training_args = TFTrainingArguments(
    output_dir = output_dir,   # output이 저장될 경로
    num_train_epochs=3,   # train 시킬 총 epochs
    per_device_train_batch_size=4,   # 각 device당 batch_size
    per_device_eval_batch_size=16,   # evaluation 시에 batch_size
    warmup_steps=500,   # learning rate scheduler에 따른 warmup_step 설정
    weight_decay=0.01,   # weight decay
    logging_dir='./logs',   # log가 저장될 경로
    do_train=True,
    do_eval=True,
    eval_steps=1000
)

max_seq_length=128
task_name="mrpc"

with training_args.strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


아래에서 생성하게 될 `TFTrainer`의 인자로 넘겨주어야 할 것 중에 `compute_metrics` 메소드가 있습니다. 이것은 task가 classification인지 regression인지에 따라 모델의 출력 형태가 달라지므로 task별로 적합한 출력 형식을 고려해 모델의 성능을 계산하는 방법을 미리 지정해 두는 것입니다.

In [78]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    if output_mode == "classification":
        preds = np.argmax(p.predictions, axis=1)
    elif output_mode == "regression":
        preds = np.squeeze(p.predictions)
    return glue_compute_metrics(task_name, preds, p.label_ids)

output_mode = glue_output_modes[task_name]
output_mode

'classification'

`TFTrainer`를 활용할 때 데이터셋에서 잊지 않아야 할 것이  
`tf.data.experimental.assert_cardinality()`를 데이터셋에 적용해 주는 것입니다. 이를 호출해 주지 않으면 `TFTrainer.train()`에서 assert fail이 발생합니다.

In [79]:
train_dataset = train_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['train'].num_examples))
validation_dataset = validation_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['validation'].num_examples))
test_dataset = test_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['test'].num_examples))

이제 `TFTrainer`를 생성해서 본격적으로 학습을 시작해 봅시다.

In [80]:
trainer = TFTrainer(
    model=model,   # 학습시킬 model
    args=training_args,   # TFTrainingArguments를 통해 설정한 arguments
    train_dataset=train_dataset,   # training dataset
    eval_dataset=validation_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,
)

In [74]:
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)

학습이 끝나면 Evaluation을 진행하여 위 `model.fit()`으로 학습한 경우와 비교해 봅시다.

In [81]:
# Evaluation
results = {}
if training_args.do_eval:
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results2.txt")
    
    with open(output_eval_file, "w") as writer:
        for key, value in result.items():
            writer.write(f"{key} = {value}\n")
            
        results.update(result)
        
# 파일에 쓴 테스트 결과 확인
!cat ~/aiffel/transformers/eval_results2.txt

/aiffel/aiffel/transformers/src/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/aiffel/aiffel/transformers/src/transformers/data/metrics/__init__.py:42: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/aiffel/aiffel/transformers/src/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datas

eval_loss = 0.5472124539888822
eval_acc = 0.7788461538461539
eval_f1 = 0.8501628664495114
eval_acc_and_f1 = 0.8145045101478326
